# Ingest data from CSV file

In [0]:
%run ../SetUp

In [0]:
csv_file_path = session_helper.get_storage_account_url(folder="raw", file="circuits.csv")
csv_file_path

In [0]:
dbutils.fs.head(csv_file_path, 200)

In [0]:
display(spark.sql(f"SELECT * FROM csv.`{csv_file_path}` LIMIT 5"))

In [0]:
circuits_df = (
    spark.read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(csv_file_path)
    .withColumnRenamed("circuitId", "circuit_id")
    .withColumnRenamed("circuitRef", "circuit_ref")
    .withColumnRenamed("lat", "latitude")
    .withColumnRenamed("long", "longitude")
    .withColumnRenamed("alt", "altitude")
    .drop("url")
)

display(circuits_df.limit(5))

In [0]:
circuits_df.write.format("delta").saveAsTable("test.circuits_bronze")

In [0]:
circuits_df.createOrReplaceTempView("circuits_df")
spark.sql("CREATE OR REPLACE TABLE test.circuits_bronze AS (SELECT * FROM circuits_df)")